# todo
  - test different:
    - augmentations
    - models
        - v3/v7
        - tiny/standard
    - dataset sizes
    - other parameters
- test if 4 of same class is equivilant to 1 class
- change from "python3.9" keywork
- move to gpu cluster
- code cleanup
- documentation
- fix class bug (1->0)

# Remember:
Restart kernel before running to reset current working directory.

# How to Train YOLOv7 on a Custom Dataset

This tutorial is based on the [YOLOv7 repository](https://github.com/WongKinYiu/yolov7) by WongKinYiu. This notebook shows training on **your own custom objects**. Many thanks to WongKinYiu and AlexeyAB for putting this repository together.


### **Accompanying Blog Post**

We recommend that you follow along in this notebook while reading the blog post on [how to train YOLOv7](https://blog.roboflow.com/yolov7-custom-dataset-training-tutorial/), concurrently.

### **Steps Covered in this Tutorial**

To train our detector we take the following steps:

* Install YOLOv7 dependencies
* Load custom dataset from Roboflow in YOLOv7 format
* Run YOLOv7 training
* Evaluate YOLOv7 performance
* Run YOLOv7 inference on test images
* OPTIONAL: Deployment
* OPTIONAL: Active Learning


### Preparing a Custom Dataset

In this tutorial, we will utilize an open source computer vision dataset from one of the 90,000+ available on [Roboflow Universe](https://universe.roboflow.com).

If you already have your own images (and, optionally, annotations), you can convert your dataset using [Roboflow](https://roboflow.com), a set of tools developers use to build better computer vision models quickly and accurately. 100k+ developers use roboflow for (automatic) annotation, converting dataset formats (like to YOLOv7), training, deploying, and improving their datasets/models.

Follow [the getting started guide here](https://docs.roboflow.com/quick-start) to create and prepare your own custom dataset.

#Install Dependencies

_(Remember to choose GPU in Runtime if not already selected. Runtime --> Change Runtime Type --> Hardware accelerator --> GPU)_

In [1]:
import os
print(os.getcwd())

/home/trymg/Dropbox/master/semester 3/IT3915 Master preparatory project


In [2]:
!python3.9 -m pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable


In [3]:
# Download YOLOv7 repository and install requirements
!git clone https://github.com/WongKinYiu/yolov7
%cd yolov7
!python3.9 -m pip install -r requirements.txt

fatal: destination path 'yolov7' already exists and is not an empty directory.
/home/trymg/Dropbox/master/semester 3/IT3915 Master preparatory project/yolov7
Defaulting to user installation because normal site-packages is not writeable


# Set parameters

In [4]:

PROJECT_NAME = "Merged-sheep-dataset"
PROJECT_VERSION = 1
PROJECT_DIRECTORY = f"{PROJECT_NAME}-{PROJECT_VERSION}"

AUGMENTATION = True
AUGMENTATIONS = float('inf')

# Download Correctly Formatted Custom Data

Next, we'll download our dataset in the right format. Use the `YOLOv7 PyTorch` export. Note that this model requires YOLO TXT annotations, a custom YAML file, and organized directories. The roboflow export writes this for us and saves it in the correct spot.


In [5]:
from roboflow import Roboflow

# print(rf.workspace().projects()) # for debugging

# custom code snippet generated from roboflow dataset using the export function

# # old dataset with 4k images:
# rf = Roboflow(api_key="Sf4Q132h8vYyzxbVfF7t")
# project = rf.workspace("it3915masterpreparatoryproject").project("sheep-detection-2")
# dataset = project.version(3).download("yolov7")

# new merged dataset with 6k images:
from roboflow import Roboflow
rf = Roboflow(api_key="Sf4Q132h8vYyzxbVfF7t")
project = rf.workspace("it3915masterpreparatoryproject").project("merged-sheep-dataset")
dataset = project.version(1).download("yolov7")


# os.rename(f"./yolov7/{PROJECT_DIRECTORY}", f"./yolov7/{PROJECT_NAME}")

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to Merged-sheep-dataset-1 in yolov7pytorch:: 100%|██████████| 6140/6140 [00:00<00:00, 13212.25it/s]


'/home/trymg/Dropbox/master/semester 3/IT3915 Master preparatory project/yolov7'

# Custom image augmentation using Albumentations

## Augmentation plot utils

In [ ]:
import random
import cv2
from matplotlib import pyplot as plt
import matplotlib.patches as patches
import numpy as np
import albumentations as A



def plot_examples(images, bboxes=None):
    fig = plt.figure(figsize=(15, 15))
    columns = 4
    rows = 5

    for i in range(1, len(images)):
        if bboxes is not None:
            img = visualize_bbox(images[i - 1], bboxes[i - 1], class_name="sheep")
        else:
            img = images[i-1]
        fig.add_subplot(rows, columns, i)
        plt.imshow(img)
    plt.show()


# From https://albumentations.ai/docs/examples/example_bboxes/
def visualize_bbox(img, bbox, class_name, color=(255, 0, 0), thickness=5):
    """Visualizes a single bounding box on the image"""
    x_min, y_min, x_max, y_max = map(int, bbox)
    # cv2.rectangle(img, (x_min, y_min), (x_max, y_max), color, thickness)
    cv2.rectangle(img, (int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])), color, thickness)
    
    return img

In [ ]:
# https://albumentations.ai/docs/examples/example_bboxes2/

def visualize(image, bboxes, category_ids, category_id_to_name):
    img = image.copy()
    for bbox, category_id in zip(bboxes, category_ids):
        class_name = category_id_to_name[category_id]
        img = visualize_bbox(img, bbox, class_name)
    plt.figure(figsize=(12, 12))
    plt.axis('off')
    plt.imshow(img)

In [ ]:
# copied from: https://www.kaggle.com/code/reighns/augmentations-data-cleaning-and-bounding-boxes/notebook

def draw_rect_with_labels(img, bboxes,class_id, class_dict={1: 'sheep'}, color=None):
    img = img.copy()
    bboxes = bboxes[:, :4]
    bboxes = bboxes.reshape(-1, 4)
    for bbox, label in zip(bboxes, class_id):
        pt1, pt2 = (bbox[0], bbox[1]), (bbox[2], bbox[3])
        pt1 = int(pt1[0]), int(pt1[1])
        pt2 = int(pt2[0]), int(pt2[1])
        class_name = class_dict[label]
        ((text_width, text_height), _) = cv2.getTextSize(class_name, cv2.FONT_HERSHEY_SIMPLEX, 0.35, 1) 
        img = cv2.rectangle(img.copy(), pt1, pt2, color, int(max(img.shape[:2]) / 200))
        img = cv2.putText(img.copy(), class_name, (int(bbox[0]), int(bbox[1]) - int(0.3 * text_height)), cv2.FONT_HERSHEY_SIMPLEX,fontScale=1,color = (255,255,255), lineType=cv2.LINE_AA)
    return img

## Augmentation classification

In [ ]:
from cmath import inf
import cv2
import albumentations as A
import numpy as np
from PIL import Image
import os, os.path
import shutil

In [ ]:
if AUGMENTATION: 
    CLASS_LABEL = '0'

    # load train images
    images = []
    images_load_path = f"./{PROJECT_DIRECTORY}/train/images/"
    valid_images = [".jpg",".gif",".png",".tga"]

    LIMIT = AUGMENTATIONS
    i = 0
    for image_file_name in os.listdir(images_load_path):
        if i == LIMIT:
            break
        ext = os.path.splitext(image_file_name)[1]
        if ext.lower() not in valid_images:
            continue
        images.append(cv2.imread(os.path.join(images_load_path, image_file_name)))
        i += 1


    # load train labels
    labels = [] # one label per image, multiple bboxes per image
    labels_load_path = f"./{PROJECT_DIRECTORY}/train/labels/"
    i = 0
    for label_file_name in os.listdir(labels_load_path):
        if i == LIMIT:
            break
        label = open(os.path.join(labels_load_path, label_file_name), "r")
        bboxes = []
        for bbox in label:
            bbox = np.array(bbox.split(' ')[1:]).astype(np.float32)
            bboxes.append(bbox)
        labels.append(bboxes)
        i += 1



    # augment images using transformation

    augmented_labels = []
    augmented_images_nd = []
    for image, label in zip(images, labels):
        label = np.array(label)

        transform = A.Compose(
            [
                A.RandomCrop(width=640, height=640),
                A.Rotate(border_mode=cv2.BORDER_CONSTANT),
                A.HorizontalFlip(),
                A.VerticalFlip(),
                A.RGBShift(r_shift_limit=25, g_shift_limit=25, b_shift_limit=25, p=0.9),
                A.OneOf([
                    A.Blur(blur_limit=3, p=0.5),
                    A.ColorJitter(p=0.5),
                ], p=1.0),
                A.RandomBrightnessContrast()
            ],
            bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels'])
        )
        
        augmentation = transform(image=image, bboxes=label, class_labels=[CLASS_LABEL]*len(label))
        augmented_img = augmentation["image"]
        augmented_label = augmentation["bboxes"]
        augmented_labels.append(augmented_label)
        # augmented_category_ids = [int(CLASS_LABEL)]*len(label)
        augmented_images_nd.append(augmented_img)
        
        # draw_image = draw_rect_with_labels(augmented_img, label, [1]*len(label))

        # visualize(
        #     augmented_img,
        #     augmented_label,
        #     augmented_category_ids,
        #     {1: 'sheep', 2: 'sheep', 3: 'sheep', 0: 'sheep'}
        # )

        # plt.figure()
        # plt.imshow(augmented_img)


        # augment_and_show(transform, image)


    # plot_examples(augmented_images_nd) # comment out to save runtime


    augmented_images = [Image.fromarray(augmented_image_nd) for augmented_image_nd in augmented_images_nd]

    # create image save dir
    img_save_dir = f"./{PROJECT_DIRECTORY}/train/images"


    # save augmented images
    for i in range(len(augmented_images)):
        image_file_name = f"{i}.png"
        image_dir = os.path.join(img_save_dir, image_file_name)
        augmented_images[i].save(image_dir)
        print(f"saved augmented image: {image_dir}")


    # create labels save dir
    labels_save_dir = f"./{PROJECT_DIRECTORY}/train/labels"

    # write augmented labels
    for i in range(len(augmented_labels)):
        label_file_name = f"{i}.txt"
        label_dir = os.path.join(labels_save_dir, label_file_name)
        with open(label_dir, 'w') as f:
            for bbox in augmented_labels[i]:
                f.write(f"{CLASS_LABEL} {bbox[0]} {bbox[1]} {bbox[2]} {bbox[3]}\n")
        print(f"saved augmented label: {label_dir}")


# Begin Custom Training

We're ready to start custom training.

NOTE: We will only modify one of the YOLOv7 training defaults in our example: `epochs`. We will adjust from 300 to 100 epochs in our example for speed. If you'd like to change other settings, see details in [our accompanying blog post](https://blog.roboflow.com/yolov7-custom-dataset-training-tutorial/).

In [ ]:
# download COCO starting checkpoint (weights)
!wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7-tiny.pt

In [ ]:
# image results from previous runs are deleted
!rm runs/train/* -rf 
!rm runs/detect/* -rf
# may need to remove labels.cache and/or images.cache beforehand?

In [ ]:
!python3.9 train.py --batch-size 16 --epochs 20 --data Merged-sheep-dataset-1/data.yaml --weights 'yolov7-tiny.pt' --device 0 

In [ ]:
# todo check if dir not empty

if AUGMENTATION: 

    # remove augmented images
    img_save_dir = f"./{PROJECT_DIRECTORY}/train/images"
    for i in range(len(augmented_images)):
        image_file_name = f"{i}.png"
        image_dir = os.path.join(img_save_dir, image_file_name)
        os.remove(os.path.join(img_save_dir, image_file_name))
        print(f"removed image: {image_dir}")

    # remove augmented labels
    labels_save_dir = f"./{PROJECT_DIRECTORY}/train/labels"
    for i in range(len(labels)):
        label_file_name = f"{i}.txt"
        label_dir = os.path.join(labels_save_dir, label_file_name)
        os.remove(os.path.join(labels_save_dir, label_file_name))
        print(f"removed label: {label_dir}")


# Evaluation

We can evaluate the performance of our custom training using the provided evalution script.

Note we can adjust the below custom arguments. For details, see [the arguments accepted by detect.py](https://github.com/WongKinYiu/yolov7/blob/main/detect.py#L154).

In [ ]:
# Run evaluation
!python3.9 detect.py --weights runs/train/exp/weights/best.pt --conf 0.3 --source Merged-sheep-dataset-1/train/images


In [ ]:
import glob
from IPython.display import Image, display

i = 0
limit = 10000 # max images to print
for imageName in glob.glob('runs/detect/exp/*.jpg'): #assuming JPG
    print(imageName)
    if i < limit:
      display(Image(filename=imageName))
      print("\n")
    i = i + 1
    

# Reparameterize for Inference

https://github.com/WongKinYiu/yolov7/blob/main/tools/reparameterization.ipynb

# OPTIONAL: Deployment

To deploy, you'll need to export your weights and save them to use later.

## export results

In [ ]:
# # optional, zip to download weights and results locally

!zip -r export.zip runs/detect
!zip -r export.zip runs/train/exp/weights/best.pt
!zip export.zip runs/train/exp/*

# OPTIONAL: Active Learning Example

Once our first training run is complete, we should use our model to help identify which images are most problematic in order to investigate, annotate, and improve our dataset (and, therefore, model).

To do that, we can execute code that automatically uploads images back to our hosted dataset if the image is a specific class or below a given confidence threshold.


In [ ]:
# # setup access to your workspace
# rf = Roboflow(api_key="YOUR_API_KEY")                               # used above to load data
# inference_project =  rf.workspace().project("YOUR_PROJECT_NAME")    # used above to load data
# model = inference_project.version(1).model

# upload_project = rf.workspace().project("YOUR_PROJECT_NAME")

# print("inference reference point: ", inference_project)
# print("upload destination: ", upload_project)

In [ ]:
# # example upload: if prediction is below a given confidence threshold, upload it 

# confidence_interval = [10,70]                                   # [lower_bound_percent, upper_bound_percent]

# for prediction in predictions:                                  # predictions list to loop through
#   if(prediction['confidence'] * 100 >= confidence_interval[0] and 
#           prediction['confidence'] * 100 <= confidence_interval[1]):
        
#           # upload on success!
#           print(' >> image uploaded!')
#           upload_project.upload(image, num_retry_uploads=3)     # upload image in question

# Next steps

Congratulations, you've trained a custom YOLOv7 model! Next, start thinking about deploying and [building an MLOps pipeline](https://docs.roboflow.com) so your model gets better the more data it sees in the wild.